In [1]:
# Step 1: Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
# Step 2: Load Dataset
df = pd.read_csv("Customer_Reviews.csv")

# ✅ Clean missing or blank values
df = df.dropna(subset=["Review_Text", "Sentiment"])
df["Sentiment"] = df["Sentiment"].str.strip()  # remove spaces

print("Dataset size:", df.shape)
print(df.head())


Dataset size: (25, 3)
   Review_ID                                        Review_Text Sentiment
0          1  The product quality is amazing and exceeded my...  Positive
1          2   Very bad experience the item broke after one use  Negative
2          3               Excellent service and quick delivery  Positive
3          4        The size was wrong and material feels cheap  Negative
4          5                  Totally worth the money I love it  Positive


In [3]:
# Step 3: Split features and labels
X = df["Review_Text"]
y = df["Sentiment"]

# Step 4: Convert text to TF-IDF (with bigrams to catch 'never buy', 'not good')
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range=(1, 2))
X_tfidf = vectorizer.fit_transform(X)

# Step 5: Split train/test data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42, stratify=y)

In [4]:
# Step 6: Train SVM Model
svm = SVC(kernel="linear", probability=True, random_state=42)
svm.fit(X_train, y_train)

# Step 7: Evaluate Model
y_pred = svm.predict(X_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

    Negative       0.40      0.50      0.44         4
    Positive       0.33      0.25      0.29         4

    accuracy                           0.38         8
   macro avg       0.37      0.38      0.37         8
weighted avg       0.37      0.38      0.37         8

Accuracy: 0.375


In [5]:
# Step 8: Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Negative", "Positive"],
            yticklabels=["Negative", "Positive"])
plt.title("Confusion Matrix - Sentiment Analysis (SVM)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [6]:
# Step 9: Test on New Reviews
sample_reviews = [
    "The product quality is excellent and works great",
    "Horrible experience I will never buy again",
    "Not worth the money at all",
    "Absolutely loved the product and quick delivery"
]
sample_features = vectorizer.transform(sample_reviews)
predictions = svm.predict(sample_features)

print("\nSample Predictions:")
for review, sentiment in zip(sample_reviews, predictions):
    print(f"'{review}' → {sentiment}")


Sample Predictions:
'The product quality is excellent and works great' → Positive
'Horrible experience I will never buy again' → Positive
'Not worth the money at all' → Negative
'Absolutely loved the product and quick delivery' → Positive
